In [1]:
#!pip install transformers sentencepiece accelerate datasets torchvision tqdm deep-translator


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel, ViTModel
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.notebook import tqdm
import pandas as pd
import os

################################# every time #################################3

2025-05-10 18:19:09.691291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746901149.873864      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746901149.929817      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Load Dataset
df = pd.read_csv("/kaggle/input/preprocessed-data-2103-new/preprocessed_2103.csv") 
df['label'] = df['final_labels_task2_1'].map({'YES': 1, 'NO': 0})

df.head()

,Unnamed: 0,id_EXIST,lang,text,meme,path_memes,number_annotators,annotators,labels_task2_1,final_labels_task2_1,label
0,0,110001,es,2+2=5 MITO Albert Einstein tenía bajo rendimie...,110001.jpeg,/kaggle/input/memes-dataset/memes/110001.jpeg,6,"['Annotator_1', 'Annotator_2', 'Annotator_3', ...","['YES', 'YES', 'YES', 'YES', 'YES', 'YES']",YES,1
1,1,110002,es,CUANDO UNA MUJER VA A LUCHAR POR SUS DERECHOS,110002.jpeg,/kaggle/input/memes-dataset/memes/110002.jpeg,6,"['Annotator_1', 'Annotator_2', 'Annotator_3', ...","['YES', 'YES', 'YES', 'YES', 'YES', 'YES']",YES,1
2,2,110003,es,ІЯ ЕГЕЯ Е MOA ¿El Partido Republicano busca pe...,110003.jpeg,/kaggle/input/memes-dataset/memes/110003.jpeg,6,"['Annotator_1', 'Annotator_2', 'Annotator_3', ...","['YES', 'YES', 'NO', 'NO', 'NO', 'NO']",NO,0
3,3,110005,es,Ya verás como este 8 de marzo hay uno que te s...,110005.jpeg,/kaggle/input/memes-dataset/memes/110005.jpeg,6,"['Annotator_1', 'Annotator_2', 'Annotator_3', ...","['NO', 'YES', 'NO', 'NO', 'YES', 'NO']",NO,0
4,4,110006,es,"Princesa, hoy es tu día, no laves ningún plato...",110006.jpeg,/kaggle/input/memes-dataset/memes/110006.jpeg,6,"['Annotator_1', 'Annotator_2', 'Annotator_3', ...","['YES', 'YES', 'YES', 'YES', 'YES', 'YES']",YES,1


In [4]:
df['label'].value_counts()

label
1    2369
0    1675
Name: count, dtype: int64

Step 2: Translate Spanish Text to English

In [5]:
from sklearn.model_selection import train_test_split

min_count = df["lang"].value_counts().min()  # Get minimum count across languages
df_balanced = df.groupby("lang").apply(lambda x: x.sample(min_count)).reset_index(drop=True)  #Ensuring Equal Samples for "en" & "es"

from sklearn.model_selection import train_test_split

# Split Data: 85% Training, 15% Validation
train_df, val_df = train_test_split(
    df_balanced, 
    test_size=0.15, 
    stratify=df_balanced["label"],   #Ensures that both training and validation sets maintain the same proportion of each label/class.
    random_state=42
)

# Check the shape of the splits
print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)


Train set shape: (3417, 11)
Validation set shape: (603, 11)


/tmp/ipykernel_31/3585815532.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby("lang").apply(lambda x: x.sample(min_count)).reset_index(drop=True)  #Ensuring Equal Samples for "en" & "es"


 Step 3: Load BLIP & BERT

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
###################################################################################### every time

In [7]:
import transformers
print(transformers.__version__)


4.51.1


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,BlipProcessor, BlipForConditionalGeneration
from torch.optim import AdamW  # correct for transformers 4.51.1



blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
bert_model = AutoModel.from_pretrained("bert-base-multilingual-cased").to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


############################################3############################################# every time

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Step 4: Prepare BERT Dataset

In [9]:
# 1a) Tokenizer for original + generated text
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def process_text(batch_texts):
    """Tokenize a list of strings for BERT."""
    if isinstance(batch_texts, str):
        batch_texts = [batch_texts]
    encoded = tokenizer(
        batch_texts,
        padding="max_length",
        truncation=True,
        max_length=96,
        return_tensors="pt"
    )
    return {k: v.to(device) for k, v in encoded.items()}

# 1b) BLIP processor for images + caption generation
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

def process_image(image_path):
    """Load & preprocess a single image for BLIP (pixel_values)."""
    img = Image.open(image_path).convert("RGB")
    return blip_processor(images=img, return_tensors="pt")["pixel_values"].squeeze(0)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
# from PIL import Image
# import torchvision.transforms as transforms
# def process_image(image_path):
#     """Loads and preprocesses the image for BLIP model."""
#     image = Image.open(image_path).convert("RGB")
#     return blip_processor(images=image, return_tensors="pt")["pixoncatenate the three vectors—(1) vision CLS, (2) caption embedding, and (3) original-text embedding—along the feature dimension.el_values"].squeeze(0)  # Remove batch dimension

In [11]:
# ─── Phase 2: Dataset & Collate fn ────────────────────────────────────────────

class MemeDataset(Dataset):
    def __init__(self, df):
        self.df = df
        # optional extra transforms (e.g., normalization) can go here
        self.resize = transforms.Resize((224, 224))
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_tensor = process_image(row["path_memes"])  # [3,H,W]
        # You could also do extra transforms: img_tensor = self.resize(img_tensor)
        text = row["text"]
        label = torch.tensor(row["label"], dtype=torch.long)
        return {"image": img_tensor.to(device),
                "text": text,
                "label": label.to(device)}

def meme_collate_fn(batch):
    images = torch.stack([item["image"] for item in batch], dim=0)  # [B,3,H,W]
    texts  = [item["text"]  for item in batch]                     # list[str]
    labels = torch.stack([item["label"] for item in batch], dim=0) # [B]
    # we delay BERT tokenization until inside model for generated captions,
    # but we still need to tokenize original texts now
    text_inputs = process_text(texts)
    return {"image": images, "text_inputs": text_inputs, "raw_texts": texts, "label": labels}

In [12]:
# ─── Phase 3: Model Definition (UPDATED) ───────────────────────────────────────

class AttentionPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        # x: [B, L, H]
        weights = torch.softmax(self.attn(x), dim=1)  # [B, L, 1]
        return torch.sum(weights * x, dim=1)          # [B, H]

class MemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        # BLIP captioner
        self.blip = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-base"
        ).to(device)
        # BERT encoder
        self.bert = AutoModel.from_pretrained("bert-base-multilingual-cased").to(device)

        # allow full fine-tuning
        for p in self.blip.parameters(): p.requires_grad = True
        for p in self.bert.parameters(): p.requires_grad = True

        # dims
        vh = self.blip.vision_model.config.hidden_size
        th = self.bert.config.hidden_size
        self.vision_hid = vh
        self.text_hid   = th

        # attention pooling for BERT outputs
        self.attn_pool = AttentionPooling(th)

        # final classifier: [ vision_cls + cap_BERT + orig_BERT ] → classes
        total_dim = vh + 2 * th
        self.classifier = nn.Sequential(
            nn.Linear(total_dim, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, images, text_inputs, raw_texts):
        # 1) Vision CLS
        vis = self.blip.vision_model(pixel_values=images).last_hidden_state
        img_feat = vis[:, 0, :]  # [B, vh]

        # 2) Caption generation (no_grad + eval, shorter beams)
        self.blip.eval()
        with torch.no_grad():
            gen_ids = self.blip.generate(
                pixel_values=images,
                max_length=16,
                num_beams=1,
                early_stopping=True
            )
        self.blip.train()

        caps = blip_processor.batch_decode(gen_ids, skip_special_tokens=True)

        # 3) BERT encode captions
        cap_inputs = tokenizer(
            caps, padding=True, truncation=True, max_length=96, return_tensors="pt"
        ).to(device)
        cap_hid  = self.bert(**cap_inputs).last_hidden_state
        cap_feat = self.attn_pool(cap_hid)

        # 4) BERT encode original meme text
        orig_hid  = self.bert(**text_inputs).last_hidden_state
        orig_feat = self.attn_pool(orig_hid)

        # 5) concat & classify
        combined = torch.cat([img_feat, cap_feat, orig_feat], dim=1)
        return self.classifier(combined)

In [13]:
# class MemeClassifier(nn.Module):
#     def __init__(self, num_classes=2):
#         super(MemeClassifier, self).__init__()
#         self.blip = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
#         self.bert = AutoModel.from_pretrained("bert-base-uncased").to(device)
        
#         # Enable fine-tuning of BLIP and BERT (Unfreeze their parameters)
#         for param in self.blip.parameters():
#             param.requires_grad = True  # 🔹 Allow BLIP fine-tuning
#         for param in self.bert.parameters():
#             param.requires_grad = True  # 🔹 Allow BERT fine-tuning

#         # Hidden sizes
#         self.vision_hidden_size = self.blip.vision_model.config.hidden_size
#         self.text_hidden_size = self.bert.config.hidden_size
        
#         # Attention pooling over BERT
#         self.attn_pooling = AttentionPooling(self.text_hidden_size)

#         # Classifier layers
#         self.fc = nn.Sequential(
#             nn.Linear(self.vision_hidden_size + self.text_hidden_size, 512),
#             nn.ReLU(),
#             nn.Dropout(0.5),
#             nn.Linear(512, num_classes)
#         )

#     def forward(self, image, text_inputs):
#         # Extract BLIP image features
#         vision_outputs = self.blip.vision_model(image)
#         image_features = vision_outputs.last_hidden_state[:, 0, :]  # Take CLS token

#         # Extract BERT text features
#         bert_output = self.bert(**text_inputs)['last_hidden_state']
#         text_features = self.attn_pooling(bert_output)

#         # Concatenate & classify
#         combined = torch.cat((image_features, text_features), dim=1)
#         logits = self.fc(combined)
#         return logits
# ######################################################################## every time #########################################################

In [14]:
# # Custom Collate Function
# def meme_collate_fn(batch):
#     images = torch.stack([item['image'] for item in batch])  # shape: [B, C, H, W]
#     texts = [item['text'] for item in batch]  # list of strings
#     labels = torch.stack([item['label'] for item in batch])  # shape: [B]

#     text_inputs = process_text(texts)  # tokenize all at once

#     return {
#         "image": images,
#         "text": text_inputs,
#         "label": labels
#     }

In [15]:
# ─── Phase 4: DataLoaders, Loss & Optimizer ─────────────────────────────────

train_loader = DataLoader(MemeDataset(train_df),
                          batch_size=8, shuffle=True,
                          collate_fn=meme_collate_fn)
val_loader   = DataLoader(MemeDataset(val_df),
                          batch_size=8, shuffle=False,
                          collate_fn=meme_collate_fn)

model = MemeClassifier(num_classes=2).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = AdamW([
    {"params": model.blip.parameters(),   "lr": 5e-6},
    {"params": model.bert.parameters(),   "lr": 5e-6},
    {"params": model.classifier.parameters(), "lr": 1e-4},
], weight_decay=0.01)


In [16]:
# train_loader = DataLoader(MemeDataset(train_df), batch_size=8, shuffle=True, collate_fn=meme_collate_fn)
# val_loader = DataLoader(MemeDataset(val_df), batch_size=8, shuffle=False, collate_fn=meme_collate_fn)


In [18]:
# ─── Phase 5: Training & Validation Loop ────────────────────────────────────

checkpoint_dir = "/kaggle/working"
os.makedirs(checkpoint_dir, exist_ok=True)

best_val_acc = 0.0

# Lists to store training & validation accuracy/loss
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

for epoch in range(1, 12):
    # — Training —
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for batch in tqdm(train_loader, desc=f"[Train] Epoch {epoch}", leave=False):
        imgs = batch["image"]
        ti   = batch["text_inputs"]
        rt   = batch["raw_texts"]
        lbls = batch["label"]

        optimizer.zero_grad()
        logits = model(imgs, ti, rt)
        loss   = criterion(logits, lbls)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()
        correct    += (logits.argmax(1) == lbls).sum().item()
        total      += lbls.size(0)

    avg_train_loss = total_loss / len(train_loader)
    train_acc      = correct / total

    train_losses.append(avg_train_loss)
    train_accuracies.append(train_acc)
    print(f"Epoch {epoch} Train Loss: {avg_train_loss:.4f} Acc: {train_acc:.2%}")

    # — Validation —
    model.eval()
    v_loss, v_corr, v_tot = 0.0, 0, 0

    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"[Val]   Epoch {epoch}", leave=False):
            logits = model(batch["image"],
                           batch["text_inputs"],
                           batch["raw_texts"])
            loss   = criterion(logits, batch["label"])

            v_loss += loss.item()
            v_corr += (logits.argmax(1) == batch["label"]).sum().item()
            v_tot  += batch["label"].size(0)

    avg_val_loss = v_loss / len(val_loader)
    val_acc      = v_corr / v_tot

    val_losses.append(avg_val_loss)
    val_accuracies.append(val_acc)
    print(f"Epoch {epoch} Val   Loss: {avg_val_loss:.4f} Acc: {val_acc:.2%}")

    # — Save only the best model so far —
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_path = os.path.join(checkpoint_dir, "best_model_task1_rank.pt")
        torch.save(model.state_dict(), best_path)
        print(f"→ New best model saved (epoch {epoch}, Acc: {val_acc:.2%})")

print("Done!")

[Train] Epoch 1:   0%|          | 0/428 [00:00<?, ?it/s]

Epoch 1 Train Loss: 0.6267 Acc: 64.24%


[Val]   Epoch 1:   0%|          | 0/76 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1 Val   Loss: 0.6248 Acc: 65.01%
→ New best model saved (epoch 1, Acc: 65.01%)


...

Epoch 10 Val   Loss: 2.4272 Acc: 64.68%


[Train] Epoch 11:   0%|          | 0/428 [00:00<?, ?it/s]

Epoch 11 Train Loss: 0.0343 Acc: 99.27%


[Val]   Epoch 11:   0%|          | 0/76 [00:00<?, ?it/s]

Epoch 11 Val   Loss: 2.9820 Acc: 62.85%
Done!


In [ ]:
# model = MemeClassifier(num_classes=2).to(device)
# criterion = nn.CrossEntropyLoss()  # Standard loss function for classification
# #optimizer = optim.AdamW(model.parameters(), lr=2e-5)  # Optimizer
# optimizer = optim.AdamW([
#     {"params": model.blip.parameters(), "lr": 5e-6},  # 🔹 Lower LR for BLIP
#     {"params": model.bert.parameters(), "lr": 5e-6},  # 🔹 Lower LR for BERT
#     {"params": model.fc.parameters(), "lr": 1e-4}     # 🔹 Higher LR for classifier
# ], weight_decay=0.01)





######################################################################## every time ##############################################################

In [1]:
import os
import json
import torch
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel, BlipProcessor, BlipForConditionalGeneration

# ─── Configuration ─────────────────────────────────────────────────────────────

CHECKPOINT = "/kaggle/input/checkpoint-task1-rank/best_model_task1_rank.pt"
CSV_PATH   = "/kaggle/input/testing-clef-testcases/testing_data_1605.csv"
BATCH_SIZE = 8
TEST_CASE  = "EXIST2025"  # if you need to change this label, set it here

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ─── Re-instantiate Models & Tokenizers ────────────────────────────────────────

# BLIP for caption generation + vision
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model     = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# BERT
bert_model  = AutoModel.from_pretrained("bert-base-multilingual-cased").to(device)
tokenizer   = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# ─── Dataset & Collate ─────────────────────────────────────────────────────────

def process_image(path):
    img = Image.open(path).convert("RGB")
    tensor = blip_processor(images=img, return_tensors="pt")["pixel_values"]
    return tensor.squeeze(0)

class TestDataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        return {
            "id":        str(row["id_EXIST"]),
            "image":     process_image(row["path_memes"]),
            "raw_text":  row["text"]
        }

def collate_fn(batch):
    images    = torch.stack([x["image"]    for x in batch], dim=0).to(device)
    raw_texts = [             x["raw_text"] for x in batch]
    # only original text needs tokenization here
    encoded   = tokenizer(raw_texts,
                          padding="max_length",
                          truncation=True,
                          max_length=96,
                          return_tensors="pt").to(device)
    ids       = [x["id"] for x in batch]
    return images, encoded, raw_texts, ids

# ─── Model Definition (same as during training) ────────────────────────────────

import torch.nn as nn

class AttentionPooling(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim, 1)
    def forward(self, x):
        weights = torch.softmax(self.attn(x), dim=1)
        return torch.sum(weights * x, dim=1)

class MemeClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.blip = BlipForConditionalGeneration.from_pretrained(
            "Salesforce/blip-image-captioning-base"
        ).to(device)
        self.bert = AutoModel.from_pretrained(
            "bert-base-multilingual-cased"
        ).to(device)
        vh = self.blip.vision_model.config.hidden_size
        th = self.bert.config.hidden_size
        self.attn_pool = AttentionPooling(th)
        self.classifier = nn.Sequential(
            nn.Linear(vh + 2*th, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, images, text_inputs, raw_texts):
        # vision features
        vis      = self.blip.vision_model(pixel_values=images).last_hidden_state
        img_feat = vis[:, 0, :]
        # generate captions
        self.blip.eval()
        with torch.no_grad():
            gen_ids = self.blip.generate(
                pixel_values=images,
                max_length=16, num_beams=1, early_stopping=True
            )
        self.blip.train()
        caps = blip_processor.batch_decode(gen_ids, skip_special_tokens=True)
        # encode captions
        cap_inputs = tokenizer(caps, padding=True, truncation=True,
                               max_length=96, return_tensors="pt").to(device)
        cap_hid    = self.bert(**cap_inputs).last_hidden_state
        cap_feat   = self.attn_pool(cap_hid)
        # encode original text
        orig_hid  = self.bert(**text_inputs).last_hidden_state
        orig_feat = self.attn_pool(orig_hid)
        # concat + classify
        combined  = torch.cat([img_feat, cap_feat, orig_feat], dim=1)
        return self.classifier(combined)

# ─── Load Checkpoint ───────────────────────────────────────────────────────────

model = MemeClassifier(num_classes=2).to(device)
model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
model.eval()

# ─── Inference ────────────────────────────────────────────────────────────────

test_ds = TestDataset(CSV_PATH)
test_loader = DataLoader(test_ds,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         collate_fn=collate_fn)

results = []
with torch.no_grad():
    for images, txt_inputs, raw_texts, ids in test_loader:
        logits = model(images, txt_inputs, raw_texts)
        preds  = logits.argmax(dim=1).cpu().tolist()
        for uid, p in zip(ids, preds):
            results.append({
                "test_case": TEST_CASE,
                "id":        uid,
                "value":     "YES" if p==1 else "NO"
            })

# ─── Dump JSON ────────────────────────────────────────────────────────────────

out_path = "/kaggle/working/predictions.json"
with open(out_path, "w") as f:
    json.dump(results, f, indent=2)

print(f"Wrote {len(results)} predictions to {out_path}")


2025-05-16 18:24:48.058301: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747419888.253792      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747419888.316456      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/tmp/ipykernel_31/400722942.py:120: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(CHECKPOINT, map_location=device))
/usr/local/lib/python3.1

Wrote 1053 predictions to /kaggle/working/predictions.json


In [2]:
import pandas as pd

# 1) Load your original test file
orig_df = pd.read_csv("/kaggle/input/testing-clef-testcases/testing_data_1605.csv")

# 2) Load the JSON predictions
preds = pd.read_json("/kaggle/input/task1-prediction/task1_rank_prediction.json")

# 3) Prepare preds: rename 'value'→'label', ensure ID types match
preds = preds.rename(columns={"value": "label", "id": "id_EXIST"})
preds["id_EXIST"] = preds["id_EXIST"].astype(orig_df["id_EXIST"].dtype)

# 4) Merge into the original DataFrame
merged = orig_df.merge(
    preds[["id_EXIST", "label"]],
    on="id_EXIST",
    how="left"
)

# 5) Save out
out_path = "testing_with_labels.csv"
merged.to_csv(out_path, index=False)
print(f"Wrote {len(merged)} rows with labels to {out_path}")


Wrote 1053 rows with labels to testing_with_labels.csv


In [ ]:
# import torch
# import os

# # Create checkpoint directory if it doesn't exist
# checkpoint_dir = '/kaggle/working'
# os.makedirs(checkpoint_dir, exist_ok=True)

# num_epochs = 25
# best_val_accuracy = 0  # Keep track of the best validation accuracy

# # Lists to store training & validation accuracy/loss
# train_losses, val_losses = [], []
# train_accuracies, val_accuracies = [], []

# for epoch in range(num_epochs):
#     ###################################
#     # -------- Training Phase --------#
#     ###################################
#     model.train()
#     train_loss, correct_train, total_train = 0, 0, 0
    
#     for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
#         images, texts, labels = batch["image"], batch["text"], batch["label"]

#         # 🔹 Ensure Correct Shape for BLIP
#         if len(images.shape) == 3:
#             images = images.unsqueeze(0)

#         # 🔹 Move Data to Device (CPU/GPU)
#         images, labels = images.to(device), labels.to(device)
#         texts = {key: val.to(device) for key, val in texts.items()}  # Move text tensors to device

#         optimizer.zero_grad()
#         outputs = model(images, texts)  # Forward Pass
#         loss = criterion(outputs, labels)  # Compute Loss
#         loss.backward()  # Backpropagation
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient Clipping
#         optimizer.step()  # Update Weights
        
#         train_loss += loss.item()
#         correct_train += (outputs.argmax(1) == labels).sum().item()
#         total_train += labels.size(0)
    
#     train_accuracy = correct_train / total_train
#     avg_train_loss = train_loss / len(train_loader)

#     train_losses.append(avg_train_loss)
#     train_accuracies.append(train_accuracy)
    
#     ######################################
#     # -------- Validation Phase --------###
#     #####################################
#     model.eval()
#     val_loss, correct_val, total_val = 0, 0, 0
    
#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
#             images, texts, labels = batch["image"], batch["text"], batch["label"]

#             # 🔹 Ensure Correct Shape for BLIP
#             if len(images.shape) == 3:
#                 images = images.unsqueeze(0)

#             # 🔹 Move Data to Device
#             images, labels = images.to(device), labels.to(device)
#             texts = {key: val.to(device) for key, val in texts.items()}  # Move text tensors to device

#             outputs = model(images, texts)
#             loss = criterion(outputs, labels)

#             val_loss += loss.item()
#             correct_val += (outputs.argmax(1) == labels).sum().item()
#             total_val += labels.size(0)

#     val_accuracy = correct_val / total_val
#     avg_val_loss = val_loss / len(val_loader)

#     val_losses.append(avg_val_loss)
#     val_accuracies.append(val_accuracy)

#     print(f"Epoch {epoch+1}: Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2%} | "
#           f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2%}")

#     # Save the model if it has the highest validation accuracy so far
#     if val_accuracy > best_val_accuracy:
#         best_val_accuracy = val_accuracy
#         best_model_path = os.path.join(checkpoint_dir, f"best_model_epoch_{epoch+1}.pt")
#         torch.save(model.state_dict(), best_model_path)
#         print(f"Saved best model with Val Acc: {val_accuracy:.2%} at epoch {epoch+1}")

# print("Training & Validation Complete!")


In [ ]:
!ls /kaggle/working/checkpoints

In [ ]:
# import shutil
# import zipfile

# # Define paths
# model_path = "/kaggle/working/checkpoints/best_model_epoch_2.pt"
# zip_path = "/kaggle/working/best_model.zip"

# # Zip the model file
# with zipfile.ZipFile(zip_path, "w") as zipf:
#     zipf.write(model_path, arcname="best_model_epoch_2.pt")

# # Move to working directory
# shutil.move(zip_path, "/kaggle/working/best_model.zip")



## End to End Pipeline

In [ ]:
# import gradio as gr
# import torch
# import torchvision.transforms as transforms
# from PIL import Image
# import pytesseract
# from transformers import AutoTokenizer


# checkpoint_path = '/kaggle/input/task1-checkpoint-blip-and-bert/best_model_epoch_2.pt'
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load(checkpoint_path, map_location=device))
# model.to(device)
# model.eval()

# # Load tokenizer for text preprocessing
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# # Define image transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# def extract_text_from_image(image):
#     """Extract text from meme image using Tesseract OCR."""
#     text = pytesseract.image_to_string(image)
#     return text.strip()

# def classify_meme(image):
#     """Classify whether a meme is sexist or not."""
#     # Extract text from the image
#     extracted_text = extract_text_from_image(image)
    
#     # Preprocess image
#     image = transform(image).unsqueeze(0).to(device)
    
#     # Tokenize text
#     encoded_text = tokenizer(extracted_text, return_tensors="pt", padding=True, truncation=True)
#     encoded_text = {key: val.to(device) for key, val in encoded_text.items()}
    
#     # Run inference
#     with torch.no_grad():
#         output = model(image, encoded_text)
#         prediction = torch.argmax(output, dim=1).item()
    
#     # Map prediction to label
#     label = "Sexist Meme" if prediction == 1 else "Not Sexist Meme"
#     return label

# # Gradio Interface
# demo = gr.Interface(
#     fn=classify_meme,
#     inputs=gr.Image(type="pil"),
#     outputs="text",
#     title="Meme Classifier",
#     description="Upload a meme to classify whether it is sexist or not.",
# )

# demo.launch(share=True)
